# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: **LoRA** - as suggested in the instructions
* Model: **GPT-2** - as suggested in the instructions
* Evaluation approach: **Huggingface Trainer: evaluate** - as suggested in the instructions
* Fine-tuning dataset: **dair-ai/emotion** - Emotion is a dataset of English Twitter messages with six basic emotions: anger, fear, joy, love, sadness, and surprise. 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 66.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 36.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import random
import numpy as np
import pandas as pd
import gc
import os
from sklearn.metrics import (accuracy_score, classification_report, precision_score, recall_score)

import torch

from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          DataCollatorWithPadding, Trainer, 
                          TrainingArguments, AutoModelForSequenceClassification
                         )
from peft import (get_peft_config, get_peft_model, LoraConfig, 
                  TaskType, PeftModel, PeftConfig, 
                  AutoPeftModelForSequenceClassification
                 )


In [3]:
splits = ["train", "test"]
dataset = {split: ds for split, ds in zip(splits, load_dataset("dair-ai/emotion", split=splits))}

dataset["train"] = dataset["train"].shuffle(seed=42).select(range(1500))
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(100))
    
# View the dataset characteristics
print(dataset["train"], dataset["test"])

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 1500
}) Dataset({
    features: ['text', 'label'],
    num_rows: 100
})


In [4]:
from collections import Counter

Counter(dataset["test"]["label"])

Counter({0: 30, 1: 38, 2: 7, 3: 13, 4: 11, 5: 1})

In [5]:
#Loading the gpt2 model from pre-trained
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Set the padding token to be the same as the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

def tokenize_dataset(dataset, splits, tokenizer):
    """Tokenizes the dataset for the specified splits.

    Args:
        dataset: The dataset containing the splits.
        splits: A list of split names (e.g., ['train', 'test']).
        tokenizer: The tokenizer to use for tokenization.

    Returns:
        A dictionary containing tokenized datasets for each split.
    """
    tokenized_ds = {}
    for split in splits:
        tokenized_ds[split] = dataset[split].map(
            lambda x: tokenizer(x['text'], padding='max_length', truncation=True, return_tensors='pt'),
            batched=True
        )
    return tokenized_ds

tokenized_dataset = tokenize_dataset(dataset, splits, tokenizer)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
num_labels = 6
id2label = {
            0: "sadness", 
            1: "joy",
            2: "love",
            3: "anger",
            4: "fear",
            5: "surprise"
             }

label2id = {v: k for k, v in id2label.items()}

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=num_labels,
    id2label=id2label, 
    label2id=label2id,
)

#freeze
for param in model.base_model.parameters():
    param.requires_grad = False
    
# set the pad token of the model's configuration
model.config.pad_token_id = model.config.eos_token_id

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=6, bias=False)
)

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall
    }



trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/emotion_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

### We then just evaluate the results

In [11]:
trainer.evaluate()

/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 3.6283998489379883,
 'eval_accuracy': 0.11,
 'eval_precision': 0.0121,
 'eval_recall': 0.11,
 'eval_runtime': 8.4988,
 'eval_samples_per_second': 11.766,
 'eval_steps_per_second': 5.883}

In [12]:
df = pd.DataFrame(tokenized_dataset["test"])
df = df[["text", "label"]]

# Replace <br /> tags in the text with spaces
df["text"] = df["text"].str.replace("<br />", " ")

# Add the model predictions to the dataframe
predictions = trainer.predict(tokenized_dataset["test"])
df["predicted_label"] = np.argmax(predictions[0], axis=1)

df.head(2)

/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,text,label,predicted_label
0,i was feeling really troubled and down over wh...,0,4
1,i feel so thrilled to have three such distingu...,1,4


In [13]:
df["predicted_label"].value_counts()

predicted_label
4    100
Name: count, dtype: int64

In [15]:
df[df["label"] != df["predicted_label"]].head(5)

,text,label,predicted_label
0,i was feeling really troubled and down over wh...,0,4
1,i feel so thrilled to have three such distingu...,1,4
2,i feel is that the most likeable characters ar...,1,4
3,i tune out the rest of the world and focus on ...,1,4
4,i sit here writing this i feel unhappy inside,0,4


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

### PEFT Model - creating setup

In [16]:
# Access the encoder layers (if applicable)
for name, layer in model.named_modules():
    print(name, layer)

 GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=6, bias=False)
)
transformer GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p

In [17]:

#Need to setup the configuration using LoraCongif
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    inference_mode=False, 
    r=8, 
    target_modules=["c_attn"],
    lora_alpha=32, 
    lora_dropout=0.03
)

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",    
    num_labels=num_labels,
    id2label=id2label, 
    label2id=label2id,
)

# creating a PEFT model
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

peft_model.config.pad_token_id = peft_model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 304,128 || all params: 124,743,936 || trainable%: 0.2438018309763771


### We apply similar logic as above. Functions were already created for accuracy

In [18]:
gc.collect()

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:126'  # Adjust the size as needed
#torch.cuda.empty_cache()


trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./data/emotion_analysis_perf",
        learning_rate=2e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.02,
        load_best_model_at_end=True,
        fp16=True
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,5.078000,2.334841,0.310000,0.150625,0.310000


/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Checkpoint destination directory ./data/emotion_analysis_perf/checkpoint-750 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=750, training_loss=4.18056201171875, metrics={'train_runtime': 220.4332, 'train_samples_per_second': 6.805, 'train_steps_per_second': 3.402, 'total_flos': 786678939648000.0, 'train_loss': 4.18056201171875, 'epoch': 1.0})

### Evaluation of PEFT model

In [20]:
trainer.evaluate()

/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 2.334841251373291,
 'eval_accuracy': 0.31,
 'eval_precision': 0.150625,
 'eval_recall': 0.31,
 'eval_runtime': 5.5741,
 'eval_samples_per_second': 17.94,
 'eval_steps_per_second': 8.97,
 'epoch': 1.0}

In [21]:
items_for_manual_review = tokenized_dataset["test"].select(
    [34, 57, 99, 25, 44, 89]
)

results = trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "sentiment": [item["label"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,sentiment,predictions,labels
0,3,1,3
1,4,1,4
2,0,3,0
3,3,1,3
4,1,1,1
5,0,1,0


### Finally saving the PEFT model

In [22]:
#Saving the trained model
peft_model.save_pretrained("./peft_model")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

### Load model

In [25]:
peft_saved = "./peft_model"
config = PeftConfig.from_pretrained(peft_saved)

In [26]:
model_peft_inf = AutoPeftModelForSequenceClassification.from_pretrained(
        peft_saved,
        num_labels=num_labels,
        id2label=id2label, 
        label2id=label2id)

model_peft_inf

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.03, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding

In [27]:
print(config)

LoraConfig(peft_type='LORA', auto_mapping=None, base_model_name_or_path='gpt2', revision=None, task_type='SEQ_CLS', inference_mode=True, r=8, target_modules=['c_attn'], lora_alpha=32, lora_dropout=0.03, fan_in_fan_out=True, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)


In [28]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

inference_dataset = tokenized_dataset["test"].select(
    [random.randint(0, 100) for _ in range(10)]
)

In [29]:
print(inference_dataset)

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10
})


In [30]:
for x in inference_dataset:
    print(x["text"])

i feel like i am the world for this boy and im glad that for a time i can be that for him
i mentioned in my last blog that i have started to get the feeling that i have been pressured into studying things i do not like which has also made me into a person i might not fully be
i tune out the rest of the world and focus on the rhythm of the needles and the softness of the yarn and for that time i feel my most peaceful
i am off on wednesday to a postgraduate open day but there will be plenty to write about the rest of the week i feel sure
i could change the emphasis and say i am stella and i m noticing i m feeling impatient
i think itd be easier if i had parents that argued with me about it then i could feel rebellious or something p but right now i just feel like a burden
i could empathize with tab because of raging hormones and the connection feeling like someone else gets you thinks youre smart pretty worth attention
im feeling insecure at the moment
i am off on wednesday to a postgrad

In [71]:
inference_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10
})

In [72]:
for x in inference_dataset:
    print(x['text'])

i feel like i am the world for this boy and im glad that for a time i can be that for him
i mentioned in my last blog that i have started to get the feeling that i have been pressured into studying things i do not like which has also made me into a person i might not fully be
i tune out the rest of the world and focus on the rhythm of the needles and the softness of the yarn and for that time i feel my most peaceful
i am off on wednesday to a postgraduate open day but there will be plenty to write about the rest of the week i feel sure
i could change the emphasis and say i am stella and i m noticing i m feeling impatient
i think itd be easier if i had parents that argued with me about it then i could feel rebellious or something p but right now i just feel like a burden
i could empathize with tab because of raging hormones and the connection feeling like someone else gets you thinks youre smart pretty worth attention
im feeling insecure at the moment
i am off on wednesday to a postgrad

In [75]:
def inference(inference_dataset, model_peft_inf):
   

    # set to evaluation mode
    model_peft_inf.eval()
    
    #Empty lists
    predicted_labels = []
    predicted_class_indexes = []
    
    
    for x in inference_dataset:
            
        #tokenize
        inputs = tokenizer(x['text'], padding='max_length', truncation=True, return_tensors='pt')
        tokenizer.pad_token = tokenizer.eos_token
        
        print(inputs)
        # Perform inference
        with torch.no_grad():  # Disable gradient calculation
            outputs = model_peft_inf(**inputs)

        # Get the logits (raw predictions)
        logits = outputs.logits
        
        # Apply softmax to get probabilities
        probabilities = torch.nn.functional.softmax(logits, dim=-1)

        # Get the predicted class (index of the maximum probability)
        predicted_class_index = torch.argmax(probabilities, dim=-1).item()

        # Map the predicted class index to the corresponding label
        predicted_label = id2label[predicted_class_index]
        
        #Append
        predicted_labels.append(predicted_label)
        predicted_class_indexes.append(predicted_class_index)
    
    df_inf = pd.DataFrame(
    {
        "text": [x["text"] for x in inference_dataset],
        "predicted_label": predicted_labels,
        "predicted_label_num": predicted_class_indexes,
        "actual_labels": [x["label"] for x in inference_dataset],
    }
)
# Show all the cell

    return df_inf

In [76]:
df_inf = inference(inference_dataset=inference_dataset, 
          model_peft_inf=model_peft_inf)

{'input_ids': tensor([[   72,  1254,   588,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[   72,  4750,   287,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[   72, 14009,   503,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[   72,   716,   572,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[   72,   714,  1487,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[   72,   892,   340,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[   72,   714, 48004,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[  320,  4203, 31955,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 

In [77]:
df_inf.head()

,text,predicted_label,predicted_label_num,actual_labels
0,i feel like i am the world for this boy and im glad that for a time i can be that for him,joy,1,1
1,i mentioned in my last blog that i have started to get the feeling that i have been pressured into studying things i do not like which has also made me into a person i might not fully be,joy,1,4
2,i tune out the rest of the world and focus on the rhythm of the needles and the softness of the yarn and for that time i feel my most peaceful,joy,1,1
3,i am off on wednesday to a postgraduate open day but there will be plenty to write about the rest of the week i feel sure,joy,1,1
4,i could change the emphasis and say i am stella and i m noticing i m feeling impatient,joy,1,3


### Evaluation

In [78]:
true_labels = list(df_inf['actual_labels'].values)
predicted_labels = list(df_inf['predicted_label_num'].values)

print(classification_report(true_labels, predicted_labels))

              precision    recall  f1-score   support

           1       0.50      1.00      0.67         5
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         2

    accuracy                           0.50        10
   macro avg       0.12      0.25      0.17        10
weighted avg       0.25      0.50      0.33        10



/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
